### Columns:

**General Columns**
* url: url of dfs
* short_description, description: Description of dfs (in English and German) written by users

**Categorical Columns**
* make_model, make, model: Model of dfs. Ex:Audi A1
* body_type, body: Body type of dfs Example: van, sedans
* vat: VAT deductible, price negotiable
* registration, first_registration: First registration date and year of dfs.
* prev_owner, previous_owners: Number of previous owners
* type: new or used
* next_inspection, inspection_new: information about inspection (inspection date,..)
* body_color, body_color_original: Color of df Ex: Black, red
* paint_type: Paint type of df Ex: Metallic, Uni/basic
* upholstery: Upholstery information (texture, color)
* gearing_type: Type of gear Ex: dfmatic, manual
* fuel : fuel type Ex: diesel, benzine
* co2_emission, emission_class, emission_label: emission information
* drive_chain: drive chain Ex: front,rear, 4WD
* consumption: consumption of df in city, country and combination (lt/100 km)
* country_version
* entertainment_media
* safety_security
* comfort_convenience
* extras

**Quantitative Columns**
* price: Price of cars
* km: km of dfs
* hp: horsepower of dfs (kW)
* displacement: displacement of dfs (cc)
* warranty: warranty period (month)
* weight: weight of df (kg)
* nr_of_doors: number of doors
* nr_of_seats : number of seats
* cylinders: number of cylinders
* gears: number of gears



# PART- 1 `( Data Cleaning )`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
from scipy import special

In [2]:
df_org = pd.read_json('./data/scout_car.json', lines=True)

In [3]:
df = df_org.copy()

In [4]:
df.head(3).T

,0,1,2
url,https://www.autoscout24.com//offers/audi-a1-sp...,https://www.autoscout24.com//offers/audi-a1-1-...,https://www.autoscout24.com//offers/audi-a1-sp...
make_model,Audi A1,Audi A1,Audi A1
short_description,Sportback 1.4 TDI S-tronic Xenon Navi Klima,1.8 TFSI sport,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...
body_type,Sedans,Sedans,Sedans
price,15770,14500,14640
vat,VAT deductible,Price negotiable,VAT deductible
km,"56,013 km","80,000 km","83,450 km"
registration,01/2016,03/2017,02/2016
prev_owner,2 previous owners,None,1 previous owner
kW,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   url                            15919 non-null  object 
 1   make_model                     15919 non-null  object 
 2   short_description              15873 non-null  object 
 3   body_type                      15859 non-null  object 
 4   price                          15919 non-null  int64  
 5   vat                            11406 non-null  object 
 6   km                             15919 non-null  object 
 7   registration                   15919 non-null  object 
 8   prev_owner                     9091 non-null   object 
 9   kW                             0 non-null      float64
 10  hp                             15919 non-null  object 
 11  Type                           15917 non-null  object 
 12  Previous Owners                9279 non-null  

## change column names

In [6]:
df.columns.str.lower().str.replace(' ', '_').str.replace('\n','').str.replace('_&','')

Index(['url', 'make_model', 'short_description', 'body_type', 'price', 'vat',
       'km', 'registration', 'prev_owner', 'kw', 'hp', 'type',
       'previous_owners', 'next_inspection', 'inspection_new', 'warranty',
       'full_service', 'non-smoking_vehicle', 'null', 'make', 'model',
       'offer_number', 'first_registration', 'body_color', 'paint_type',
       'body_color_original', 'upholstery', 'body', 'nr._of_doors',
       'nr._of_seats', 'model_code', 'gearing_type', 'displacement',
       'cylinders', 'weight', 'drive_chain', 'fuel', 'consumption',
       'co2_emission', 'emission_class', 'comfort_convenience',
       'entertainment_media', 'extras', 'safety_security', 'description',
       'emission_label', 'gears', 'country_version', 'electricity_consumption',
       'last_service_date', 'other_fuel_types', 'availability',
       'last_timing_belt_service_date', 'available_from'],
      dtype='object')

In [7]:
df.columns = df.columns.str.lower().str.replace(' ','_').str.replace('.','').str.replace('\n','').str.replace('_&','')

df.columns

Index(['url', 'make_model', 'short_description', 'body_type', 'price', 'vat',
       'km', 'registration', 'prev_owner', 'kw', 'hp', 'type',
       'previous_owners', 'next_inspection', 'inspection_new', 'warranty',
       'full_service', 'non-smoking_vehicle', 'null', 'make', 'model',
       'offer_number', 'first_registration', 'body_color', 'paint_type',
       'body_color_original', 'upholstery', 'body', 'nr_of_doors',
       'nr_of_seats', 'model_code', 'gearing_type', 'displacement',
       'cylinders', 'weight', 'drive_chain', 'fuel', 'consumption',
       'co2_emission', 'emission_class', 'comfort_convenience',
       'entertainment_media', 'extras', 'safety_security', 'description',
       'emission_label', 'gears', 'country_version', 'electricity_consumption',
       'last_service_date', 'other_fuel_types', 'availability',
       'last_timing_belt_service_date', 'available_from'],
      dtype='object')

## url column

In [8]:
df.drop('url', axis=1, inplace=True)

# make_model column

In [9]:
df['make_model'].value_counts(dropna=False)

make_model
Audi A3           3097
Audi A1           2614
Opel Insignia     2598
Opel Astra        2526
Opel Corsa        2219
Renault Clio      1839
Renault Espace     991
Renault Duster      34
Audi A2              1
Name: count, dtype: int64

## displacement column

In [10]:
df['displacement'].value_counts(dropna=False)

displacement
[\n1,598 cc\n]    4761
[\n999 cc\n]      2438
[\n1,398 cc\n]    1314
[\n1,399 cc\n]     749
[\n1,229 cc\n]     677
                  ... 
[\n122 cc\n]         1
[\n1,198 cc\n]       1
[\n1,195 cc\n]       1
[\n2,967 cc\n]       1
[\n1,800 cc\n]       1
Name: count, Length: 78, dtype: int64

In [11]:
df['displacement'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

0        1422.0
1        1798.0
2        1598.0
3        1422.0
4        1422.0
          ...  
15914    1997.0
15915    1798.0
15916    1997.0
15917    1997.0
15918    1798.0
Name: displacement, Length: 15919, dtype: float64

In [12]:
#df['displacement'].str[0].str.replace(',','').str.replace('\n','').str.replace(' cc','').value_counts().index

In [13]:
df['displacement'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

0        1422.0
1        1798.0
2        1598.0
3        1422.0
4        1422.0
          ...  
15914    1997.0
15915    1798.0
15916    1997.0
15917    1997.0
15918    1798.0
Name: displacement, Length: 15919, dtype: float64

In [14]:
df['displacement'].str[0].str.replace('\n','').str.replace(' cc','').str.replace(',','').astype(float)

0        1422.0
1        1798.0
2        1598.0
3        1422.0
4        1422.0
          ...  
15914    1997.0
15915    1798.0
15916    1997.0
15917    1997.0
15918    1798.0
Name: displacement, Length: 15919, dtype: float64

In [15]:
df['displacement'] = df['displacement'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

In [16]:
df['displacement'].value_counts(dropna=False).index.sort_values()

Index([    1.0,     2.0,    54.0,   122.0,   139.0,   140.0,   160.0,   890.0,
         898.0,   899.0,   900.0,   929.0,   973.0,   995.0,   996.0,   997.0,
         998.0,   999.0,  1000.0,  1100.0,  1149.0,  1195.0,  1197.0,  1198.0,
        1199.0,  1200.0,  1229.0,  1239.0,  1248.0,  1300.0,  1333.0,  1350.0,
        1364.0,  1368.0,  1369.0,  1390.0,  1395.0,  1396.0,  1397.0,  1398.0,
        1399.0,  1400.0,  1422.0,  1461.0,  1490.0,  1495.0,  1496.0,  1498.0,
        1499.0,  1500.0,  1533.0,  1568.0,  1580.0,  1584.0,  1589.0,  1596.0,
        1598.0,  1599.0,  1600.0,  1618.0,  1686.0,  1696.0,  1798.0,  1800.0,
        1856.0,  1896.0,  1956.0,  1968.0,  1984.0,  1995.0,  1997.0,  1998.0,
        2000.0,  2480.0,  2967.0, 15898.0, 16000.0,     nan],
      dtype='float64', name='displacement')

## Short description.

In [17]:
df['short_description']

0              Sportback 1.4 TDI S-tronic Xenon Navi Klima
1                                           1.8 TFSI sport
2        Sportback 1.6 TDI S tronic Einparkhilfe plus+m...
3                                  1.4 TDi Design S tronic
4        Sportback 1.4 TDI S-Tronic S-Line Ext. admired...
                               ...                        
15914                Blue dCi 200CV EDC Executive 4Control
15915                  TCe 225 EDC GPF LIM Deluxe Pano,RFK
15916    Blue dCi 200 EDC Initiale Paris Leder LED Navi...
15917    Blue dCi 200CV EDC Business , NUOVA DA IMMATRI...
15918          INITIALE PARIS TCe 225 EDC GPF NAVI SHZ AUX
Name: short_description, Length: 15919, dtype: object

In [18]:
df['short_description'].str.findall('\d\.\d').str[0].astype(float)

0        1.4
1        1.8
2        1.6
3        1.4
4        1.4
        ... 
15914    NaN
15915    NaN
15916    NaN
15917    NaN
15918    NaN
Name: short_description, Length: 15919, dtype: float64

In [19]:
sd_disp = df['short_description'].str.findall('\d\.\d').str[0].astype(float)

In [20]:
df['sd_disp'] = sd_disp*1000

In [21]:
df['sd_disp'] = df['sd_disp'].replace(1600,1598).replace(1800,1798)

In [22]:
df.loc[df['sd_disp']<800,'sd_disp'] = np.nan

In [23]:
df.displacement.value_counts()

displacement
1598.0    4761
999.0     2438
1398.0    1314
1399.0     749
1229.0     677
          ... 
122.0        1
1198.0       1
1195.0       1
2967.0       1
1800.0       1
Name: count, Length: 77, dtype: int64

In [24]:
def disp(d1,d2):
    if (d1>4000) | (d1<700) | np.isnan(d1):
        if np.isnan(d2):
            return d1
        else:
            return d2
    else:
        return d1

In [25]:
df.apply(lambda x: disp(x['displacement'],x['sd_disp']),axis=1)

0        1422.0
1        1798.0
2        1598.0
3        1422.0
4        1422.0
          ...  
15914    1997.0
15915    1798.0
15916    1997.0
15917    1997.0
15918    1798.0
Length: 15919, dtype: float64

In [26]:
df['displacement'] = df.apply(lambda x: disp(x['displacement'],x['sd_disp']),axis=1)
    

In [27]:
df[['displacement', 'sd_disp']].value_counts()

displacement  sd_disp
1598.0        1598.0     3677
999.0         1000.0     1238
1398.0        1400.0     1004
1399.0        1400.0      558
1229.0        1200.0      541
                         ... 
999.0         5500.0        1
              6100.0        1
1398.0        1200.0        1
999.0         7800.0        1
9800.0        9800.0        1
Name: count, Length: 134, dtype: int64

* The displacement values in short description column was extracted and this values compared and matched with displacement column. Some null values of displacement column were filled in this way.

In [28]:
df.drop(['short_description', 'sd_disp'], axis=1, inplace=True)

In [29]:
df['displacement'].value_counts(dropna=False)

displacement
1598.0    4889
999.0     2438
1398.0    1314
1399.0     749
1229.0     677
          ... 
1390.0       1
1198.0       1
2800.0       1
2967.0       1
1800.0       1
Name: count, Length: 78, dtype: int64

## body_type column

In [30]:
df['body_type'].value_counts(dropna=False)

body_type
Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
None               60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

## Price Column

In [31]:
df['price'].sort_values()

8594        13
8828       120
6066       255
8829       331
8827      4950
         ...  
15828    64332
3587     64900
3649     67600
15826    68320
3648     74600
Name: price, Length: 15919, dtype: int64

In [32]:
df[df['price']<1000].T

,6066,8594,8828,8829
make_model,Opel Astra,Opel Corsa,Opel Corsa,Opel Corsa
body_type,Station wagon,Sedans,Compact,Other
price,255,13,120,331
vat,None,None,None,None
km,"5,563 km",123 km,12 km,10 km
registration,06/2018,06/2018,01/2019,01/2019
prev_owner,None,None,None,None
kw,NaN,NaN,NaN,NaN
hp,100 kW,66 kW,66 kW,66 kW
type,"[, Used, , Diesel (Particulate Filter)]","[, Used, , Gasoline]","[, New, , Gasoline]","[, New, , Gasoline]"


In [33]:
df[df['price']<4000].index

Index([6066, 8594, 8828, 8829], dtype='int64')

In [34]:
price_outlier = df[df['price']<4000].index

In [35]:
df.drop(price_outlier, inplace=True)
# We drop 4 rows since their price and km are meaningless

In [36]:
df.price.sort_values()

8827      4950
8825      4990
8826      5250
8824      5300
13770     5445
         ...  
15828    64332
3587     64900
3649     67600
15826    68320
3648     74600
Name: price, Length: 15915, dtype: int64

## vat column 

In [37]:
df.vat.value_counts(dropna=False)

vat
VAT deductible      10980
None                 4509
Price negotiable      426
Name: count, dtype: int64

## km column

In [38]:
df.km.value_counts(dropna=False)

km
10 km        1044
- km         1024
1 km          367
5 km          170
50 km         148
             ... 
43,197 km       1
10,027 km       1
35,882 km       1
57,840 km       1
57 km           1
Name: count, Length: 6689, dtype: int64

In [39]:
df[df['km'] == '- km'].registration.value_counts()

registration
-/-        838
02/2019     32
06/2019     26
04/2019     25
03/2019     23
05/2019     22
01/2019     17
06/2018      6
12/2018      6
08/2018      6
07/2018      5
02/2018      4
05/2018      3
07/2019      2
04/2018      2
10/2018      2
09/2018      1
12/2017      1
01/2018      1
11/2018      1
03/2018      1
Name: count, dtype: int64

In [40]:
df.km.str.replace(',','').str.findall('\d+').str[0].astype(float)

0        56013.0
1        80000.0
2        83450.0
3        73000.0
4        16200.0
          ...   
15914        NaN
15915     9900.0
15916       15.0
15917       10.0
15918        NaN
Name: km, Length: 15915, dtype: float64

In [41]:
df.km = df.km.str.replace(',','').str.findall('\d+').str[0].astype(float)

In [42]:
df[df.km<100].km.value_counts()

km
10.0    1044
1.0      367
5.0      170
50.0     148
15.0     109
        ... 
36.0       1
65.0       1
59.0       1
18.0       1
57.0       1
Name: count, Length: 66, dtype: int64

In [43]:
df[df.km<100].km.count()

2362

* km column is cleaned and converted to float. '-' rows are converted to np.nan. 
* There are 2362 cars with km less than 100. These cars will be controlled after cleaning. If the year of these cars do not match with km, it can be filled  in another way.

## registration column

In [44]:
df.registration.value_counts(dropna=False)

registration
-/-        1597
03/2018     695
02/2019     585
05/2018     572
03/2019     543
04/2018     541
02/2018     539
01/2019     539
03/2016     536
04/2016     532
06/2018     530
01/2018     511
04/2019     506
02/2016     472
03/2017     471
05/2016     459
06/2016     452
05/2019     440
06/2017     409
05/2017     404
07/2018     396
04/2017     380
01/2016     376
02/2017     368
01/2017     306
08/2018     285
06/2019     224
07/2017     215
11/2017     180
07/2016     176
10/2016     160
10/2017     154
09/2017     149
11/2016     142
09/2018     141
09/2016     141
12/2016     134
12/2017     123
08/2017     114
11/2018     110
12/2018     103
10/2018      97
08/2016      94
07/2019       6
09/2019       5
08/2019       1
11/2019       1
12/2019       1
Name: count, dtype: int64

In [45]:
df.registration = df.registration.replace('-/-',np.nan)

In [46]:
df.registration.value_counts(dropna=False)

registration
NaN        1597
03/2018     695
02/2019     585
05/2018     572
03/2019     543
04/2018     541
02/2018     539
01/2019     539
03/2016     536
04/2016     532
06/2018     530
01/2018     511
04/2019     506
02/2016     472
03/2017     471
05/2016     459
06/2016     452
05/2019     440
06/2017     409
05/2017     404
07/2018     396
04/2017     380
01/2016     376
02/2017     368
01/2017     306
08/2018     285
06/2019     224
07/2017     215
11/2017     180
07/2016     176
10/2016     160
10/2017     154
09/2017     149
11/2016     142
09/2018     141
09/2016     141
12/2016     134
12/2017     123
08/2017     114
11/2018     110
12/2018     103
10/2018      97
08/2016      94
07/2019       6
09/2019       5
08/2019       1
11/2019       1
12/2019       1
Name: count, dtype: int64

In [47]:
pd.to_datetime(df.registration)

/tmp/ipykernel_18354/4176771207.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df.registration)


0       2016-01-01
1       2017-03-01
2       2016-02-01
3       2016-08-01
4       2016-05-01
           ...    
15914          NaT
15915   2019-01-01
15916   2019-03-01
15917   2019-06-01
15918   2019-01-01
Name: registration, Length: 15915, dtype: datetime64[ns]

In [48]:
pd.DatetimeIndex(df['registration']).year

Index([2016.0, 2017.0, 2016.0, 2016.0, 2016.0, 2016.0, 2016.0, 2016.0, 2016.0,
       2017.0,
       ...
       2019.0, 2019.0, 2019.0,    nan, 2019.0,    nan, 2019.0, 2019.0, 2019.0,
       2019.0],
      dtype='float64', name='registration', length=15915)

In [49]:
df['year'] = pd.DatetimeIndex(df['registration']).year 

In [50]:
df['month'] = pd.DatetimeIndex(df['registration']).month 

In [51]:
df[['registration','year','month']]

,registration,year,month
0,01/2016,2016.0,1.0
1,03/2017,2017.0,3.0
2,02/2016,2016.0,2.0
3,08/2016,2016.0,8.0
4,05/2016,2016.0,5.0
...,...,...,...
15914,NaN,NaN,NaN
15915,01/2019,2019.0,1.0
15916,03/2019,2019.0,3.0
15917,06/2019,2019.0,6.0


In [52]:
df.registration.isnull().sum()

1597

In [53]:
df.drop('registration', axis=1, inplace=True)

* registration column was converted to datetime. 
* Also year and month columns were created from registration column. 
* All null values are converted to np.nan

## prev_owner and Previous Owners columns

In [54]:
df.prev_owner.value_counts(dropna=False)

prev_owner
1 previous owner     8294
None                 6824
2 previous owners     778
3 previous owners      17
4 previous owners       2
Name: count, dtype: int64

In [55]:
df.prev_owner.str[0]

0           2
1        None
2           1
3           1
4           1
         ... 
15914    None
15915       1
15916       1
15917    None
15918       1
Name: prev_owner, Length: 15915, dtype: object

In [56]:
df.prev_owner = df.prev_owner.str[0].astype('float')

In [57]:
df.prev_owner.value_counts(dropna=False)

prev_owner
1.0    8294
NaN    6824
2.0     778
3.0      17
4.0       2
Name: count, dtype: int64

### previous_owners column

In [58]:
df['previous_owners'].value_counts(dropna=False)

previous_owners
\n1\n                                                                                        8101
NaN                                                                                          6636
\n2\n                                                                                         766
\n0\n                                                                                         163
\n3\n                                                                                          17
                                                                                             ... 
[\n1\n, \n111 g CO2/km (comb)\n]                                                                1
[\n1\n, \nEuro 6\n]                                                                             1
[\n2\n, \n119 g CO2/km (comb)\n]                                                                1
[\n1\n, \n, 6.6 l/100 km (comb), \n, 8.1 l/100 km (city), \n, 5.7 l/100 km (country), \n]       1
[\n2

In [59]:
df['previous_owners'].str.findall('\d+').str[0].value_counts(dropna=False)

previous_owners
1      8101
NaN    6866
2       766
0       163
3        17
4         2
Name: count, dtype: int64

In [60]:
df['previous_owners'] = df['previous_owners'].str.findall('\d+').str[0].astype('float')

In [61]:
df['previous_owners'].value_counts(dropna=False)

previous_owners
1.0    8101
NaN    6866
2.0     766
0.0     163
3.0      17
4.0       2
Name: count, dtype: int64

In [62]:
(df['previous_owners']-df['prev_owner']).value_counts(dropna=False)

0.0    8886
NaN    7029
Name: count, dtype: int64

### Combine these two columns by apply method

In [63]:
def prev_owner_combine(p1,p2):
    if p1 == p2:
        return p1
    elif np.isnan(p1) :
        if np.isnan(p2):
            return np.nan
        else:
            return p2
    elif np.isnan(p2):
        if np.isnan(p1):
            return np.nan
        else:
            return p1
    else:
        return 'conflict'

In [64]:
df.apply(lambda x: prev_owner_combine(x['prev_owner'],x['previous_owners']), axis=1)

0        2.0
1        NaN
2        1.0
3        1.0
4        1.0
        ... 
15914    NaN
15915    1.0
15916    1.0
15917    NaN
15918    1.0
Length: 15915, dtype: float64

In [65]:
df['prev_owner'] = df.apply(lambda x: prev_owner_combine(x['prev_owner'],x['previous_owners']), axis=1)

In [66]:
df['prev_owner'].value_counts(dropna=False)

prev_owner
1.0    8294
NaN    6661
2.0     778
0.0     163
3.0      17
4.0       2
Name: count, dtype: int64

In [67]:
df.drop(['previous_owners'],axis=1, inplace=True)

* prev_owner and Previous Owners columns were combined and Previous Owners column was dropped.

## kW column

In [68]:
df.drop('kw',axis=1,inplace=True)

## hp column

In [69]:
df.hp.str[:-3]

0         66
1        141
2         85
3         66
4         66
        ... 
15914    147
15915    165
15916    146
15917    147
15918    165
Name: hp, Length: 15915, dtype: object

In [70]:
df.hp = df.hp.str.findall('\d+').str[0].astype('float')

In [71]:
#df.hp = df.hp.str[:-3].replace('-',np.nan).astype('float')

In [72]:
df.hp.value_counts(dropna=False)

hp
85.0     2542
66.0     2119
81.0     1402
100.0    1307
110.0    1112
         ... 
84.0        1
195.0       1
44.0        1
239.0       1
9.0         1
Name: count, Length: 81, dtype: int64

In [73]:
df.hp.isnull().sum()

88

## type column

In [74]:
df['new_used'] = df.type.str[1]

In [75]:
df.new_used.value_counts(dropna=False)

new_used
Used              11094
New                1648
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: count, dtype: int64

* new_used column was created. This column consists of the information about if the car is new, used, pre-registered, demonstration or employee's car

In [76]:
df.type.str[0].value_counts(dropna=False)

type
       15913
NaN        2
Name: count, dtype: int64

In [77]:
df.type.str[2].value_counts(dropna=False)

type
       15913
NaN        2
Name: count, dtype: int64

In [78]:
df.type.str[3].value_counts(dropna=False)

type
Diesel (Particulate Filter)                                                                                  4314
Super 95                                                                                                     3338
Gasoline                                                                                                     3172
Diesel                                                                                                       2982
Super 95 / Regular/Benzine 91                                                                                 424
                                                                                                             ... 
Regular/Benzine 91 / Super 95 / Regular/Benzine E10 91                                                          1
Super Plus 98 / Super E10 95                                                                                    1
Super 95 / Regular/Benzine 91 / Super Plus 98 / Regular/Benzine E10 91 / Super Plus

In [79]:
df['fuel_type'] = df.type.str[3]

In [80]:
benzine = df.type.str[3].str.contains('Benzine', na=False, regex=True)

In [81]:
df['fuel_type'][benzine].value_counts()

fuel_type
Super 95 / Regular/Benzine 91                                                                                                     424
Regular/Benzine 91                                                                                                                354
Regular/Benzine 91 (Particulate Filter)                                                                                           100
Regular/Benzine 91 / Super 95 / Super Plus 98 / Regular/Benzine E10 91 / Super E10 95 / Super Plus E10 98                          13
Regular/Benzine E10 91                                                                                                             11
Regular/Benzine 91 / Regular/Benzine E10 91                                                                                         7
Super 95 / Regular/Benzine 91 / Super Plus 98 / Regular/Benzine E10 91 / Super E10 95 / Super Plus E10 98                           7
Regular/Benzine 91 / Super 95 / Super Plus 98 / Regu

In [82]:
particulate = df.type.str[3].str.contains('Particulate', na=False, regex=True)

In [83]:
df['particulate']='unparticulate'

In [84]:
df.loc[particulate,'particulate']='particulate'

In [85]:
df['particulate'].value_counts()

particulate
unparticulate    11097
particulate       4818
Name: count, dtype: int64

In [86]:
df.loc[benzine,'fuel_type'] = 'benzine'

In [87]:
df['fuel_type'][df['fuel_type'] == 'benzine'].value_counts()

fuel_type
benzine    950
Name: count, dtype: int64

In [88]:
super = df.type.str[3].str.contains('Super', na=False, regex=True)

In [89]:
gasoline = df.type.str[3].str.contains('Gasoline', na=False, regex=True)

In [90]:
df.loc[super,'fuel_type'] = 'benzine'

In [91]:
df.loc[gasoline,'fuel_type'] = 'benzine'

In [92]:
df['fuel_type'].value_counts()

fuel_type
benzine                        8546
Diesel (Particulate Filter)    4314
Diesel                         2982
CNG                              33
LPG                              16
Liquid petroleum gas (LPG)       10
Others                            5
CNG (Particulate Filter)          3
Others (Particulate Filter)       1
Biogas                            1
Domestic gas H                    1
Electric                          1
Name: count, dtype: int64

In [93]:
gas = df['fuel_type'].isin(['LPG','Liquid petroleum gas (LPG)',\
                              'CNG','CNG (Particulate Filter)',\
                              'Biogas','Domestic gas H'])
          

In [94]:
df.loc[gas,'fuel_type'] = 'gas'

In [95]:
df['fuel_type'].value_counts()

fuel_type
benzine                        8546
Diesel (Particulate Filter)    4314
Diesel                         2982
gas                              64
Others                            5
Others (Particulate Filter)       1
Electric                          1
Name: count, dtype: int64

In [96]:
diesel = df['fuel_type'].isin(['Diesel (Particulate Filter)', 'Diesel'])

In [97]:
df.loc[diesel,'fuel_type'] = 'diesel'

In [98]:
others = df['fuel_type'].isin(['Others', 'Others (Particulate Filter)', 'Electric'])

In [99]:
df.loc[others,'fuel_type'] = 'others'

In [100]:
df['fuel_type'].value_counts(dropna=False)

fuel_type
benzine    8546
diesel     7296
gas          64
others        7
NaN           2
Name: count, dtype: int64

In [101]:
df.drop('type',axis=1, inplace=True)

* fuel_type column was cleaned. Another column 'particulate' that shows 'Particulate Filter' of cars was created. 

## Next Inspection', 'Inspection new' columns

In [102]:
df["next_inspection"].str[0].str.replace("\n","").value_counts(dropna=False)

next_inspection
NaN        12380
             710
06/2021      437
03/2021      172
05/2021      153
           ...  
12/2017        1
01/2023        1
01/1921        1
11/2022        1
03/2017        1
Name: count, Length: 70, dtype: int64

In [103]:
listINS=[]
for i in df["next_inspection"]:
    if type(i)==float:
        listINS.append(i)
    elif type(i)==list:
        listINS.append(i[0].strip())
    else:
        listINS.append(i.replace("\n",""))

In [104]:
df['next_inspection_date'] = listINS

In [105]:
df['next_inspection_date'] = pd.to_datetime(df['next_inspection_date'])

## Next inspection date column was created.

/tmp/ipykernel_18354/1802073081.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['next_inspection_date'] = pd.to_datetime(df['next_inspection_date'])


In [106]:
df['next_inspection'].str[1].str.replace('\n','')

0         99 g CO2/km (comb)
1                        NaN
2                        NaN
3                        NaN
4                        NaN
                ...         
15914                    NaN
15915    168 g CO2/km (comb)
15916                    NaN
15917                    NaN
15918                    NaN
Name: next_inspection, Length: 15915, dtype: object

In [107]:
df['car_emission'] = df['next_inspection'].str[1].str.replace('\n','').str[:-16]

In [108]:
df['car_emission'] = df['car_emission'].replace('',np.nan)

In [109]:
df['car_emission'][df['car_emission'].str.isdecimal()==False]

463      0 k
473      0 k
973      0 k
1008     0 k
1077     0 k
1232     0 k
1294     0 k
1654     0 k
1655     0 k
1734     0 k
1741     0 k
1880     0 k
2026     0 k
2054     0 k
2055     0 k
2235     0 k
2346     0 k
2694     0 k
2953     0 k
3534     0 k
5420     0 k
5621     0 k
7715     0 k
7716     0 k
9538     0 k
9543     0 k
9900     0 k
9901     0 k
10121    0 k
10292    0 k
10509    0 k
10517    0 k
12517    0 k
13222    0 k
15750    0 k
Name: car_emission, dtype: object

In [110]:
df['car_emission'] = df['car_emission'].replace(' 0 k',0)

In [111]:
df['car_emission'] = df['car_emission'].replace('0 k',0)

In [112]:
df['car_emission'] = df['car_emission'].astype('float')

In [113]:
df['car_emission'].isnull().sum()

13571

In [114]:
df.drop('next_inspection', axis=1, inplace=True)

* Car emission column was created. It can be dropped.

In [115]:
df.drop('car_emission', axis=1, inplace=True)

## 'Inspection new' column

In [116]:
def inspection(a):
    if type(a)== list:
        return a[0].replace('\n', '')
    elif type(a)== str:
        return a.replace('\n', '')
    else:
        return a

In [117]:
df['inspection_new'] = df['inspection_new'].apply(inspection)

* Inspection new: Other parts of this column shows fuel consumption. Since they are the same data in the following consumption column, this part of data was not extracted.

In [118]:
df['inspection_new'].str[2].value_counts()

inspection_new
s    3932
Name: count, dtype: int64

In [119]:
#df['fuel_cons_comb'] = df['inspection_new'].str[2].str[:-16]

#df['fuel_cons_comb'] = df['fuel_cons_comb'].replace('',np.nan).astype('float')

In [120]:
#df['fuel_cons_city'] = df['inspection_new'].str[4].str[:-16]

In [121]:
#df['fuel_cons_city'] = df['fuel_cons_city'].replace('',np.nan).astype('float')

In [122]:
#df['fuel_cons_country'] = df['inspection_new'].str[6].str[:-16]

#df['fuel_cons_country'] = df['fuel_cons_city'].replace('',np.nan).astype('float')

## Warranty column

In [123]:
import re
def clean_warranty(a):
    if type(a) == list:
        b = re.findall(r'\d+', a[0])
        if len(b)== 0:
            return np.nan
        else:
            return b[0]
    elif type(a) ==str:
        b = re.findall(r'\d+', a)
        if len(b)== 0:
            return np.nan
        else:
            return b[0]
    else:
        return a

In [124]:
df['warranty'] = df['warranty'].apply(clean_warranty)

In [125]:
df['warranty'] = df['warranty'].astype('float')

* In warranty column other parts of column are meaningless.

## Full Service

In [126]:
df['full_service'].str[2].value_counts(dropna=False)

full_service
NaN                           9685
\n4 (Green)\n                 2235
\nEuro 6\n                    2097
\nEuro 6d-TEMP\n               399
\n120 g CO2/km (comb)\n        105
                              ... 
\n253 g CO2/km (comb)\n          1
\n164 g CO2/km (comb)\n          1
\n13,983 g CO2/km (comb)\n       1
\n197 g CO2/km (comb)\n          1
\nEuro 4\n                       1
Name: count, Length: 94, dtype: int64

In [127]:
df.drop('full_service', axis=1, inplace=True)

### In this column there is no meaningful data about service. It can be dropped.

## 'Non-smoking Vehicle' column

In [128]:
df['non-smoking_vehicle'].str[0].value_counts()

non-smoking_vehicle
\n    7177
Name: count, dtype: int64

In [129]:
df.drop('non-smoking_vehicle', axis=1, inplace=True)

### In this column there is no meaningful data about service. It can be dropped.

## 'null' column

In [130]:
df.drop('null', axis=1, inplace=True)

## 'Make' column

In [131]:
df['make'] = df['make'].str.replace('\n','')

In [132]:
df.drop('make', axis=1, inplace=True)

### This column includes only main models. Match with make_model column. It can be dropped.

## 'Model' column

In [133]:
df['model'] = df['model'].str[1]

In [134]:
df.drop('model', axis=1, inplace=True)

### Also we can drop this column. It includes only models

## Offer Number column

In [135]:
df['offer_number'].str[0].value_counts()

offer_number
\nLT67679\n                                     27
\nHM53619\n                                     27
\nJV03654\n                                     27
\nXT61649\n                                     27
\nUN89904\n                                     27
                                                ..
\n74223\n                                        1
\n160_dcbb6c3e-a6da-43a3-8754-ccd994cec93b\n     1
\n6701576\n                                      1
\n6701569\n                                      1
\nRe_30000008029\n                               1
Name: count, Length: 11438, dtype: int64

In [136]:
df.drop('offer_number',axis=1,inplace=True)

### Offer number column includes only ids about columns. It can be dropped.

## First Registration column

In [137]:
df['first_registration'] = df['first_registration'].str[1].astype('float')

In [138]:
### Compare the results with year.

In [139]:
df['first_registration'].value_counts(dropna=False)

first_registration
2018.0    4520
2016.0    3674
2017.0    3273
2019.0    2851
NaN       1597
Name: count, dtype: int64

In [140]:
df['year'].value_counts(dropna=False)

year
2018.0    4520
2016.0    3674
2017.0    3273
2019.0    2851
NaN       1597
Name: count, dtype: int64

In [141]:
df.apply(lambda x: prev_owner_combine(x['first_registration'],x['year']), axis=1).value_counts(dropna=False)

2018.0    4520
2016.0    3674
2017.0    3273
2019.0    2851
NaN       1597
Name: count, dtype: int64

In [142]:
df.drop('first_registration',axis=1, inplace=True)

* Year and 'First Registration' columns are same. It can be dropped.

## 'Body Color' column

In [143]:
df['body_color'] = df['body_color'].str[1]

In [144]:
df['body_color'].value_counts(dropna=False)

body_color
Black     3745
Grey      3504
White     3405
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Yellow      51
Violet      18
Bronze       6
Orange       3
Name: count, dtype: int64

## 'Paint Type' column

In [145]:
df['paint_type'] = df['paint_type'].str[0].str[1:-1]

In [146]:
df['paint_type'].value_counts(dropna=False)

paint_type
Metallic       9793
NaN            5769
Uni/basic       347
Perl effect       6
Name: count, dtype: int64

## body_color_original column

In [147]:
df['body_color_original'] = df['body_color_original'].str[0].str[1:-1]

In [148]:
df['body_color_original'].value_counts(dropna=False)

body_color_original
NaN                              3757
Onyx Schwarz                      338
Bianco                            282
Mythosschwarz Metallic            238
Brillantschwarz                   216
                                 ... 
Rouge-Braun (G0Y)                   1
VARI COLRI DISPONIBILI              1
Kokosnussbraun Metallic             1
Farbe frei wählbar                  1
Perlmutt-Weiß Metallic (Weiß)       1
Name: count, Length: 1927, dtype: int64

In [149]:
df['body_color_original'].isnull().sum()

3757

In [150]:
#import statsmodels.api as sm
#from statsmodels.formula.api import ols
#model = ols('price ~ C(body_color_original)', data=df).fit()
#anova_table = sm.stats.anova_lm(model, typ=2)
#anova_table

### This column also can be dropped

In [151]:
df.drop('body_color_original',axis=1,inplace=True)

## upholstery column

In [152]:
df['upholstery_material'] = df['upholstery'].str[0].str.replace('\n','').str.split(', ').str[0]

In [153]:
list_color = ['Black','Grey','Brown','Beige', 'Blue', 'White']
for i in list_color:
    df['upholstery_material'] = df['upholstery_material'].replace(i,np.nan)

In [154]:
df['upholstery_material'].value_counts(dropna=False)

upholstery_material
Cloth           8421
NaN             4501
Part leather    1499
Full leather    1009
Other            368
Velour            60
alcantara         57
Name: count, dtype: int64

In [155]:
df['upholstery_color'] = df['upholstery'].str[0].str.replace('\n','').str.replace(', ','')

In [156]:
list_uph_mat = ['Cloth', 'Part leather', 'Full leather', 'Other', 'Velour', 'alcantara']
for i in list_uph_mat:
    df['upholstery_color'] = df['upholstery_color'].str.replace(i,'')

In [157]:
df['upholstery_color'] = df['upholstery_color'].replace('',np.nan)

In [158]:
df['upholstery_color'].value_counts(dropna=False)

upholstery_color
Black     8199
NaN       6036
Grey      1376
Brown      207
Beige       54
Blue        16
White       13
Red          9
Yellow       4
Orange       1
Name: count, dtype: int64

### upholstery column cleaned and by this column two columns called upholstery_color and upholstery_material were created. upholstery column can be dropped.

In [159]:
df.drop('upholstery', axis=1,inplace=True)

## body column 

In [160]:
df['body'] = df['body'].str[1]

In [161]:
df['body'].value_counts(dropna=False)

body
Sedans           7902
Station wagon    3552
Compact          3152
Van               783
Other             289
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

### This column match with body_type column

In [162]:
df['body_type'].value_counts(dropna=False)

body_type
Sedans           7902
Station wagon    3552
Compact          3152
Van               783
Other             289
Transporter        88
None               60
Off-Road           56
Coupe              25
Convertible         8
Name: count, dtype: int64

In [163]:
df[(~(df['body']==df['body_type']))][['body','body_type']]

,body,body_type
3175,NaN,None
3255,NaN,None
3975,NaN,None
3997,NaN,None
4206,NaN,None
4297,NaN,None
4298,NaN,None
5718,NaN,None
5938,NaN,None
5940,NaN,None


In [164]:
df.drop('body', axis=1,inplace=True)

### Since body and body_type columns exactly match, body column was dropped.

## nr_of_doors column

In [165]:
df['nr_of_doors'] = df['nr_of_doors'].str[0].str.replace('\n','').astype(float)

In [166]:
df['nr_of_doors'].value_counts(dropna=False)

nr_of_doors
5.0    11572
4.0     3079
3.0      831
2.0      219
NaN      212
1.0        1
7.0        1
Name: count, dtype: int64

## nr_of_seats column

In [167]:
df['nr_of_seats'] = df['nr_of_seats'].str[0].str.replace('\n','').astype(float)

In [168]:
df['nr_of_seats'].value_counts(dropna=False)

nr_of_seats
5.0    13332
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: count, dtype: int64

## model_code column

In [169]:
df['model_code'] = df['model_code'].str[0].str.replace('\n','')

In [170]:
df.drop('model_code',axis=1,inplace=True)

### This column can be dropped.

## gearing_type column

In [171]:
df['gearing_type'] = df['gearing_type'].str[1]

In [172]:
df['gearing_type'].value_counts(dropna=False)

gearing_type
Manual            8149
Automatic         7297
Semi-automatic     469
Name: count, dtype: int64

## cylinders column

In [173]:
df['cylinders'] = df['cylinders'].str[0].str.replace('\n','')

In [174]:
df['cylinders'].value_counts(dropna=False)

cylinders
4      8103
NaN    5680
3      2102
5        22
6         3
8         2
2         2
1         1
Name: count, dtype: int64

## weight column

In [175]:
df['weight'] = df['weight'].str[0].str.replace(',','').str.findall('\d+').str[0].astype('float')

In [176]:
df['weight'][df['weight']<700]

1143     102.0
8355       1.0
12779      1.0
Name: weight, dtype: float64

## drive_chain column

In [177]:
df['drive_chain'] = df['drive_chain'].str[0].str.replace('\n','')

In [178]:
df['drive_chain'].value_counts(dropna=False)

drive_chain
front    8886
NaN      6854
4WD       171
rear        4
Name: count, dtype: int64

## fuel column

In [179]:
fuel = df['fuel'].str[1]

In [180]:
particulate = fuel.str.contains('Particulate')

In [181]:
df.particulate[~particulate].value_counts()

particulate
unparticulate    11097
Name: count, dtype: int64

In [182]:
benzine = fuel.str.contains('Benzine')

In [183]:
fuel[benzine] = 'benzine'

In [184]:
df.fuel_type[benzine].value_counts()

fuel_type
benzine    950
Name: count, dtype: int64

In [185]:
super = fuel.str.contains('Super')

In [186]:
fuel[super] = 'benzine'

In [187]:
gasoline = fuel.str.contains('Gasoline')

In [188]:
fuel[gasoline] = 'benzine'

In [189]:
diesel = fuel.str.contains('Diesel')
df.fuel_type[diesel].value_counts()

fuel_type
diesel    7296
Name: count, dtype: int64

In [190]:
fuel[diesel] = 'diesel'

In [191]:
fuel.value_counts(dropna=False)

fuel
benzine                        8546
diesel                         7298
CNG                              33
LPG                              16
Liquid petroleum gas (LPG)       10
Others                            5
CNG (Particulate Filter)          3
Others (Particulate Filter)       1
Biogas                            1
Domestic gas H                    1
Electric                          1
Name: count, dtype: int64

In [192]:
gas = fuel.isin(['LPG','Liquid petroleum gas (LPG)',\
                              'CNG','CNG (Particulate Filter)',\
                              'Biogas','Domestic gas H'])

In [193]:
fuel[gas] = 'gas'

In [194]:
fuel.value_counts(dropna=False)

fuel
benzine                        8546
diesel                         7298
gas                              64
Others                            5
Others (Particulate Filter)       1
Electric                          1
Name: count, dtype: int64

In [195]:
df.drop('fuel', axis=1, inplace=True)

* fuel column totally match with fuel_type column. It was controlled and then dropped.

## Consumption

In [196]:
def consume_combined(a):
    if type(a) == list:
        if len(a) >3:
            for i in a:
                if 'comb' in i:
                    return i
        else:
            return a[0]            
    
    else:
        return a
    
df['consumption_comb'] = df['consumption'].apply(consume_combined)

In [197]:
def cleaning_consumption(a):
    if type(a) == list:
        if len(a) > 0:
            b = re.findall("\d\.?\d?", a[0])
            return b[0]
        else:
            return np.nan
    elif type(a) == str:
        b = re.findall("\d\.?\d?",a)
        return b[0]        
    else:
        return a

In [198]:
def consume_city(a):
    if type(a) == list:
        if len(a) >3:
            for i in a:
                if 'city' in i:
                    return i
        else:
            return a[1]           
    
    else:
        return a
    
df['consumption_city'] = df['consumption'].apply(consume_city)


In [199]:
def consume_country(a):
    if type(a)== list:
        if len(a) >3:
            for i in a:
                if 'country' in i:
                    return i
        else:
            return a[2]            
    
    else:
        return a
    
df['consumption_country'] = df['consumption'].apply(consume_country)

In [200]:

df['consumption_comb'] = df['consumption_comb'].apply(cleaning_consumption).astype('float')
df['consumption_city'] = df['consumption_city'].apply(cleaning_consumption).astype('float')
df['consumption_country'] = df['consumption_country'].apply(cleaning_consumption).astype('float')

df.drop("consumption",axis=1,inplace=True)

## co2_emission column

In [201]:
df['co2_emission'] = df['co2_emission'].str[0].str.findall("\d+").str[0].astype('float')

In [202]:
df['co2_emission'].value_counts(dropna=False)

co2_emission
NaN      2435
120.0     740
99.0      545
97.0      537
104.0     501
         ... 
51.0        1
165.0       1
331.0       1
80.0        1
193.0       1
Name: count, Length: 120, dtype: int64

In [203]:
# df.drop('co2_emission',axis=1,inplace=True)

## emission class column

In [204]:
df['emission_class'] = df['emission_class'].str[0].str.replace('\n','')

In [205]:
df['emission_class'].value_counts(dropna=False)

emission_class
Euro 6          10137
NaN              3627
Euro 6d-TEMP     1844
Euro 6c           127
Euro 5             78
Euro 6d            62
Euro 4             40
Name: count, dtype: int64

In [206]:
df['emission_class'].replace(['Euro 6','Euro 6d-TEMP','Euro 6d', 'Euro 6c'], 'Euro 6', inplace = True)

## comfort_convenience column

In [207]:
df['comfort_convenience'] = df['comfort_convenience'].astype('str').str.replace('[','').str.replace("]",'')

In [208]:
df['comfort_convenience'].astype('str').str.replace('[','').str.replace("]",'').str.get_dummies(sep=",")

,'Air suspension','Armrest','Automatic climate control','Auxiliary heating','Cruise control','Electric Starter','Electric tailgate','Electrical side mirrors','Electrically adjustable seats','Electrically heated windshield',...,'Leather steering wheel','Light sensor','Multi-function steering wheel','Navigation system','Panorama roof','Park Distance Control','Power windows','Rain sensor','Sunroof',nan
0,0,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15914,0,0,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
15915,0,0,1,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
15916,0,1,1,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
15917,0,0,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### This column was not changed as it will be transformed with getdummy function later

## entertainment_media column

In [209]:
df['entertainment_media']

0        [Bluetooth, Hands-free equipment, On-board com...
1        [Bluetooth, Hands-free equipment, On-board com...
2                                 [MP3, On-board computer]
3        [Bluetooth, CD player, Hands-free equipment, M...
4        [Bluetooth, CD player, Hands-free equipment, M...
                               ...                        
15914    [Bluetooth, Digital radio, Hands-free equipmen...
15915    [Bluetooth, Digital radio, Hands-free equipmen...
15916    [Bluetooth, Hands-free equipment, On-board com...
15917               [Bluetooth, Digital radio, Radio, USB]
15918                                                [USB]
Name: entertainment_media, Length: 15915, dtype: object

In [210]:
df['entertainment_media'] = df['entertainment_media'].astype('str').str.replace('[','').str.replace("]",'')

### This column was not changed as it will be transformed with getdummy function later

## extras column

In [211]:
df['extras'].astype('str').str.replace('[','').str.replace("]",'').str.get_dummies(sep=', ').sum()

'Alloy wheels'           11291
'Cab or rented Car'        310
'Catalytic Converter'     2258
'Handicapped enabled'       52
'Right hand drive'           3
'Roof rack'               2646
'Shift paddles'            508
'Ski bag'                  247
'Sliding door'               3
'Sport package'           1197
'Sport seats'             3098
'Sport suspension'        1618
'Touch screen'            4043
'Trailer hitch'            654
'Tuned car'                 13
'Voice Control'           4326
'Winter tyres'             246
nan                       2962
dtype: int64

In [212]:
df['extras'] = df['extras'].astype('str').str.replace('[','').str.replace("]",'')

### This column was not changed as it will be transformed with getdummy function later

## safety_security column

In [213]:
df['safety_security'] = df['safety_security'].astype('str').str.replace('[','').str.replace("]",'')

### This column was not changed as it will be transformed with getdummy function later

## description column

In [214]:
df['description']

0        [\n, Sicherheit:,  , Deaktivierung für Beifahr...
1        [\nLangstreckenfahrzeug daher die hohe Kilomet...
2        [\n, Fahrzeug-Nummer: AM-95365,  , Ehem. UPE 2...
3        [\nAudi A1: , - 1e eigenaar , - Perfecte staat...
4        [\n, Technik & Sicherheit:, Xenon plus, Klimaa...
                               ...                        
15914    [\nVettura visionabile nella sede in Via Roma ...
15915    [\nDach: Panorama-Glas-Schiebedach, Lackierung...
15916    [\n, Getriebe:,  Automatik, Technik:,  Bordcom...
15917    [\nDEK:[2691331], Renault Espace Blue dCi 200C...
15918    [\n, Sicherheit Airbags:,  , Seitenairbag,  , ...
Name: description, Length: 15915, dtype: object

In [215]:
df.drop('description',axis=1,inplace=True)

### This column was dropped since it includes German description of car written by users

## emission_label column

In [216]:
df['emission_label'].value_counts(dropna=False)


emission_label
NaN                     11930
[\n4 (Green)\n]          3553
[\n1 (No sticker)\n]      381
[[], [], []]               40
[\n5 (Blue)\n]              8
[\n3 (Yellow)\n]            2
[\n2 (Red)\n]               1
Name: count, dtype: int64

In [217]:
df['emission_label'] = df['emission_label'].str[0].str.findall('\((.*?)\)').str[0]

In [218]:
df['emission_label'].value_counts(dropna=False)

emission_label
NaN           11970
Green          3553
No sticker      381
Blue              8
Yellow            2
Red               1
Name: count, dtype: int64

In [219]:
df.drop('emission_label',axis=1,inplace=True)

## gears column

In [220]:
df['gears'] = df['gears'].str[0].str.findall("\d+").str[0].astype('float')

In [221]:
df['gears'].value_counts(dropna=False)

gears
6.0     5819
NaN     4712
5.0     3238
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: count, dtype: int64

## country_version column

In [222]:
df['country_version'] = df['country_version'].str[0].str.replace('\n','')

In [223]:
df['country_version'].value_counts(dropna=False)

country_version
NaN               8331
Germany           4502
Italy             1038
European Union     507
Netherlands        462
Spain              325
Belgium            314
Austria            208
Czech Republic      52
Poland              49
France              38
Denmark             33
Hungary             28
Japan                8
Slovakia             4
Croatia              4
Sweden               3
Romania              2
Bulgaria             2
Luxembourg           1
Switzerland          1
Slovenia             1
Egypt                1
Serbia               1
Name: count, dtype: int64

In [224]:
### This column can be dropped.

In [225]:
df.drop('country_version',axis=1,inplace=True)

## electricity_consumption column

In [226]:
df.loc[df['electricity_consumption'].isnull()==False, 'electricity_consumption'] = 1

In [227]:
df.loc[df['electricity_consumption'].isnull()==True, 'electricity_consumption'] = 0

In [228]:
df['electricity_consumption'].value_counts(dropna=False)

electricity_consumption
0    15778
1      137
Name: count, dtype: int64

## last_service_date column

In [229]:
df['last_service_date'] = pd.to_datetime(df['last_service_date'].str[0].str.replace('\n','').replace('',np.nan))

/tmp/ipykernel_18354/1709576087.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['last_service_date'] = pd.to_datetime(df['last_service_date'].str[0].str.replace('\n','').replace('',np.nan))


In [230]:
df['last_service_date'].value_counts(dropna=False, normalize=True)

last_service_date
NaT           0.968206
2019-05-01    0.003456
2019-01-01    0.003079
2019-02-01    0.002890
2019-06-01    0.002890
2019-04-01    0.002765
2019-03-01    0.002639
2018-12-01    0.001571
2018-10-01    0.001382
2018-05-01    0.001257
2018-01-01    0.001194
2018-06-01    0.001068
2018-07-01    0.001068
2018-09-01    0.000943
2018-11-01    0.000943
2018-04-01    0.000880
2018-08-01    0.000691
2018-03-01    0.000566
2017-06-01    0.000440
2018-02-01    0.000251
2017-05-01    0.000189
2017-02-01    0.000189
2017-10-01    0.000189
2017-01-01    0.000189
2017-12-01    0.000189
2016-04-01    0.000126
2016-06-01    0.000126
2017-11-01    0.000126
2019-11-01    0.000063
2019-10-01    0.000063
2017-07-01    0.000063
2016-03-01    0.000063
2019-08-01    0.000063
2019-09-01    0.000063
2016-05-01    0.000063
2017-04-01    0.000063
Name: proportion, dtype: float64

### This column can be dropped since 96.8% is null.

In [231]:
df.drop('last_service_date',axis=1,inplace=True)

## other_fuel_types column

In [232]:
df['other_fuel_types'].value_counts(dropna=False)

other_fuel_types
NaN             15035
[[], [], []]      880
Name: count, dtype: int64

In [233]:
df.drop('other_fuel_types',axis=1,inplace=True)

## availability column

In [234]:
df['availability'] = df['availability'].str.findall('\d+')

In [235]:
df.drop('availability',axis=1,inplace=True)

## last_timing_belt_service_date column

In [236]:
df['last_timing_belt_service_date'].str[0].value_counts(dropna=False)

last_timing_belt_service_date
NaN            15899
\n04/2016\n        2
\n12/1900\n        1
\n07/2018\n        1
\n01/1900\n        1
\n05/2019\n        1
\n09/2018\n        1
\n05/2018\n        1
\n06/2017\n        1
\n01/2019\n        1
\n02/2019\n        1
\n02/2018\n        1
\n06/2019\n        1
\n01/2018\n        1
\n04/2019\n        1
\n01/1970\n        1
Name: count, dtype: int64

In [237]:
df.drop('last_timing_belt_service_date',axis=1,inplace=True)

## available_from column

In [238]:
df['available_from'].value_counts(dropna=False)

available_from
NaN             15643
\n31/08/19\n       98
\n30/07/19\n       25
\n15/08/19\n       24
\n16/09/19\n       14
\n29/06/19\n       12
\n01/07/19\n       11
\n28/06/19\n       10
\n19/08/19\n        7
\n10/09/19\n        6
\n26/06/19\n        5
\n27/06/19\n        5
\n10/11/19\n        5
\n08/07/19\n        4
\n15/07/19\n        3
\n25/10/19\n        3
\n01/08/19\n        3
\n04/07/19\n        2
\n05/08/19\n        2
\n24/08/19\n        2
\n16/07/19\n        2
\n20/07/19\n        2
\n05/07/19\n        2
\n30/06/19\n        2
\n30/09/19\n        1
\n31/10/19\n        1
\n27/07/19\n        1
\n02/07/19\n        1
\n10/07/19\n        1
\n19/07/19\n        1
\n24/07/19\n        1
\n29/07/19\n        1
\n29/09/19\n        1
\n18/08/19\n        1
\n14/09/19\n        1
\n18/07/19\n        1
\n16/08/19\n        1
\n24/09/19\n        1
\n17/08/19\n        1
\n10/10/19\n        1
\n06/12/19\n        1
\n03/12/19\n        1
\n05/12/19\n        1
\n11/08/19\n        1
\n10/12/19\n     

In [239]:
df.drop('available_from',axis=1,inplace=True)

In [240]:
df.columns

Index(['make_model', 'body_type', 'price', 'vat', 'km', 'prev_owner', 'hp',
       'inspection_new', 'warranty', 'body_color', 'paint_type', 'nr_of_doors',
       'nr_of_seats', 'gearing_type', 'displacement', 'cylinders', 'weight',
       'drive_chain', 'co2_emission', 'emission_class', 'comfort_convenience',
       'entertainment_media', 'extras', 'safety_security', 'gears',
       'electricity_consumption', 'year', 'month', 'new_used', 'fuel_type',
       'particulate', 'next_inspection_date', 'upholstery_material',
       'upholstery_color', 'consumption_comb', 'consumption_city',
       'consumption_country'],
      dtype='object')

In [241]:
# Also vat, next_inspection_date, month, particulate columns can be deleted. 
# After filling null values consumption_city, consumption_country columns can be deleted. 

In [242]:
df.isnull().sum()

make_model                     0
body_type                     60
price                          0
vat                         4509
km                          1024
prev_owner                  6661
hp                            88
inspection_new             11983
warranty                   11062
body_color                   597
paint_type                  5769
nr_of_doors                  212
nr_of_seats                  977
gearing_type                   0
displacement                 180
cylinders                   5680
weight                      6974
drive_chain                 6854
co2_emission                2435
emission_class              3627
comfort_convenience            0
entertainment_media            0
extras                         0
safety_security                0
gears                       4712
electricity_consumption        0
year                        1597
month                       1597
new_used                       2
fuel_type                      2
particulat

In [243]:
## Month, particulate, upholstery color columns dropped.

In [244]:
df.drop(['month','particulate', 'upholstery_color'], axis=1, inplace=True)

In [245]:
df.to_csv("./data/scout_cleaned_2.csv", index=False)